# Read Data from External Location S3 Bucket

In [0]:
s3_path = "s3://external-storage-db/"
dbutils.fs.ls(s3_path)


In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(days=1)
end_date = date.today()

## Load Data into S3 bucket from the Earthquake API

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"
try:
    response = requests.get(url)
    data = response.json().get('features',[])
    if not data:
        print("Data not available for this date range!")
    else:
        dbutils.fs.put(f"{s3_path}{start_date}_earthquake_bronze.json", json.dumps(data), True)
        print("Data saved to S3!")
        dbutils.fs.ls(s3_path)
except Exception as e:
    print(f"Error: {e}")

In [0]:
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
}

dbutils.jobs.taskValues.set(key="output_data", value=output_data)